## Práctica Calificada 1

Integrantes:


### Ejercicio 1

#### Import libreries

In [1]:
# Manipulating matrixes and DataFrames
import numpy as np
import pandas as pd

# Pre-build models
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import scale

#### Load and process data

In [2]:
# Load the hitters dataset
hitters = pd.read_csv('https://raw.githubusercontent.com/qlabpucp/datasets/main/datasets/boston.csv')

# Drop string columns
hitters = hitters.drop(columns=['Unnamed: 0'])

# Dop missing values
hitters = hitters.dropna()

# Splitting into input and target variables
x = hitters.drop('crim', axis=1)
y = hitters['crim']

# Scaling data
x_scaled = scale(x)
y_scaled = scale(y)

#### Item a) Estimate coefficients

##### Fit a linear regression model

In [3]:
model = LinearRegression(fit_intercept=False)
model.fit(x_scaled, y_scaled)

LinearRegression(fit_intercept=False)

##### Estimate coefficients

In [4]:
print(model.coef_)

[ 0.12393939 -0.04653842 -0.02437248 -0.1341459   0.05137256 -0.00277602
 -0.24780411  0.6199926  -0.07397933 -0.07653287  0.1152332  -0.23529275]


In [5]:
coefs_df = pd.DataFrame(data=model.coef_, columns=["Coefs - Attribute"], index=x.columns)
coefs_df

,Coefs - Attribute
zn,0.123939
indus,-0.046538
chas,-0.024372
nox,-0.134146
rm,0.051373
age,-0.002776
dis,-0.247804
rad,0.619993
tax,-0.073979
ptratio,-0.076533


#### Item b) Estimate coefficients manually: min RSS

In [6]:
Xtx = np.dot(x_scaled.T, x_scaled)
Xty = np.dot(x_scaled.T, y_scaled)
beta = np.linalg.solve(Xtx, Xty)
print(beta)

[ 0.12393939 -0.04653842 -0.02437248 -0.1341459   0.05137256 -0.00277602
 -0.24780411  0.6199926  -0.07397933 -0.07653287  0.1152332  -0.23529275]


* En la siguiente tabla se muestra la comparación de los coficientes calculados automáticamente por la librería LinearRegression (coefs-attribute) y los resultados obtenidos del cálculo manual mediante la minimización del RSS (coefs-beta). En resumen, los resultados obtenidos por ambos métodos son relativamente idénticos.

In [7]:
coefs_df['Coefs - Beta'] = beta
coefs_df

,Coefs - Attribute,Coefs - Beta
zn,0.123939,0.123939
indus,-0.046538,-0.046538
chas,-0.024372,-0.024372
nox,-0.134146,-0.134146
rm,0.051373,0.051373
age,-0.002776,-0.002776
dis,-0.247804,-0.247804
rad,0.619993,0.619993
tax,-0.073979,-0.073979
ptratio,-0.076533,-0.076533


#### Item c) Estimate coefficients using gradient descent

In [8]:
# Initialize weights randomly
w0 = np.random.uniform(size=x_scaled.shape[1])

# Set the learning rate
alpha = 0.0002

w1 = w0.copy()

# Set a loop that will continue untl break condition is met
while True:

    # Calculate predictions
    predictions = np.dot(x_scaled, w0)
    # Calculate errors
    errors = y_scaled - predictions
    # Calculate gradient (direction to adjust weights to minimize the loss function)
    gradient = -2 * np.dot(x_scaled.T, errors)
    # Update weights
    w1 = w0 - alpha * gradient

    # Check for convergence
    if np.allclose(w1, w0, atol=1e-4):
        break

    # Prepare for the next iteration
    w0 = w1.copy()

# Print final weights
print(w1)

[ 0.12293479 -0.04928114 -0.02412288 -0.13324734  0.05154902 -0.00326229
 -0.2474964   0.61327436 -0.06607693 -0.07598548  0.11631236 -0.23400247]


In [9]:
coefs_df['Coefs - GD'] = w1
coefs_df

,Coefs - Attribute,Coefs - Beta,Coefs - GD
zn,0.123939,0.123939,0.122935
indus,-0.046538,-0.046538,-0.049281
chas,-0.024372,-0.024372,-0.024123
nox,-0.134146,-0.134146,-0.133247
rm,0.051373,0.051373,0.051549
age,-0.002776,-0.002776,-0.003262
dis,-0.247804,-0.247804,-0.247496
rad,0.619993,0.619993,0.613274
tax,-0.073979,-0.073979,-0.066077
ptratio,-0.076533,-0.076533,-0.075985


In [10]:
# Initialize weights randomly
w0 = np.random.uniform(size=x_scaled.shape[1])

# Set the learning rate
alpha = 0.00015

w2 = w0.copy()

# Set a loop that will continue untl break condition is met
while True:

    # Calculate predictions
    predictions = np.dot(x_scaled, w0)
    # Calculate errors
    errors = y_scaled - predictions
    # Calculate gradient (direction to adjust weights to minimize the loss function)
    gradient = -2 * np.dot(x_scaled.T, errors)
    # Update weights
    w2 = w0 - alpha * gradient

    # Check for convergence
    if np.allclose(w2, w0, atol=1e-5):
        break

    # Prepare for the next iteration
    w0 = w2.copy()

# Print final weights
print(w2)

[ 0.12379986 -0.04692527 -0.02433662 -0.13402661  0.05139786 -0.00284356
 -0.24776824  0.61904639 -0.07286407 -0.07645827  0.11538101 -0.23511749]


* Observamos que a medida que la tasa de convergencia y el umbral de convergencia sean más pequeños (ver coefs-GD-2) los resultados se acercan más a las estimaciones en los puntos a y b.

In [11]:
coefs_df['Coefs - GD-2'] = w2
coefs_df

,Coefs - Attribute,Coefs - Beta,Coefs - GD,Coefs - GD-2
zn,0.123939,0.123939,0.122935,0.123800
indus,-0.046538,-0.046538,-0.049281,-0.046925
chas,-0.024372,-0.024372,-0.024123,-0.024337
nox,-0.134146,-0.134146,-0.133247,-0.134027
rm,0.051373,0.051373,0.051549,0.051398
age,-0.002776,-0.002776,-0.003262,-0.002844
dis,-0.247804,-0.247804,-0.247496,-0.247768
rad,0.619993,0.619993,0.613274,0.619046
tax,-0.073979,-0.073979,-0.066077,-0.072864
ptratio,-0.076533,-0.076533,-0.075985,-0.076458
